In [1]:
import cv2
import glob
import os

def load_points_from_file(file_path):
    """
    指定のtxtファイルから (frame, track, x, y, attr) のタプルを読み込みます。
    """
    points = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(',')
            if len(parts) < 5:
                continue
            frame = int(parts[0])
            track = int(parts[1])
            x = float(parts[2])
            y = float(parts[3])
            attr = parts[4]
            points.append((frame, track, x, y, attr))
    return points

def scale_points_swapped(points, img_width, draw_height):
    """
    座標のxとyを入れ替え、固定キャリブレーション値に基づいて線形マッピングします。
    
    - 元のy座標（基準値1505）を [0, img_width] に変換 → 新しいx座標
    - 元のx座標（基準値950）を [0, draw_height] に変換 → 新しいy座標
    
    ※ はみ出す場合は倍率がそのまま適用されます。
    """
    scaled_points = []
    for point in points:
        frame, track, x, y, attr = point
        new_x = int((y / 1505) * img_width)
        new_y = int((x / 950) * draw_height)
        scaled_points.append((frame, track, new_x, new_y, attr))
    return scaled_points

def get_color(track_id):
    """
    track id毎に固定の色（BGR）を返します。  
    track idが1,2,3,5,6の場合は20%暗くして文字が読み取りやすいように調整します。
    """
    base_colors = {
        1: (0, 0, 255),         # 赤
        2: (255, 0, 0),         # 青
        3: (180, 105, 255),     # ピンク
        4: (0, 0, 0),           # 黒
        5: (0, 150, 255),       # オレンジ
        6: (136, 255, 221)      # R221 G255 B136（BGR表記）
    }
    color = base_colors.get(track_id, (0, 255, 0))
    if track_id in (1, 2, 3, 5, 6):
        factor = 0.8  # 20%暗くする
        color = tuple(int(c * factor) for c in color)
    return color

def build_attribute_color_mapping_from_file(gt_file):
    """
    指定のground_truth側のtxtファイルから、属性(attr)→色のマッピング辞書を作成します。  
    同じ属性が出現した場合、最初に出現したtrack idの色を採用します。
    """
    mapping = {}
    pts = load_points_from_file(gt_file)
    for p in pts:
        _, track, _, _, attr = p
        if attr not in mapping:
            mapping[attr] = get_color(track)
    return mapping

def draw_points(image, points):
    """
    画像上に各点を大きめの円と拡大された属性テキストで描画します。
    """
    for point in points:
        frame, track, x, y, attr = point
        color = get_color(track)
        # 円の半径を8に、塗りつぶし(-1)
        cv2.circle(image, (x, y), radius=20, color=color, thickness=-1)
        # テキストのフォントスケール1.5、太さ2で描画
        cv2.putText(image, attr, (x + 10, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, thickness=3)
    return image

def draw_points_with_mapping(image, points, attr_color_mapping):
    """
    画像上に各点を描画します。  
    各点の属性(attr)が attr_color_mapping に存在する場合は、その色で描画し、
    なければ従来の get_color(track_id) による色で描画します。  
    円の半径は8、テキストはフォントスケール1.0、太さ2で描画します。
    """
    for point in points:
        frame, track, x, y, attr = point
        if attr in attr_color_mapping:
            color = attr_color_mapping[attr]
        else:
            color = get_color(track)
        cv2.circle(image, (x, y), radius=20, color=color, thickness=-1)
        cv2.putText(image, attr, (x + 10, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, thickness=3)
    return image


In [ ]:
# フォルダのパス設定
GT_dir = '../../ground_truth/Indoor/transformed_MOT_with_attributes'
base_image_path = '../../court_images/Indoor.jpg'
output_dir_GT = '../../videos/Indoor/minimap/ground_truth'
os.makedirs(output_dir_GT, exist_ok=True)

# 背景画像の読み込みとリサイズ（強制的に縦1050×横1505）
base_image = cv2.imread(base_image_path)
if base_image is None:
    print(f"背景画像の読み込みに失敗しました: {base_image_path}")
else:
    resized_image = cv2.resize(base_image, (1505, 1050))
    height, width = resized_image.shape[:2]  # height=1050, width=1505
    # 下部100ピクセルは余白として残すので、描画領域は上部 (height - 100) = 950
    draw_height = height - 100
    print(f"画像サイズ: width={width}, height={height}, 描画領域: {draw_height}px")
    
    txt_files = glob.glob(os.path.join(GT_dir, '*.txt'))
    if not txt_files:
        print("指定フォルダにtxtファイルが見つかりませんでした:", GT_dir)
    
    for txt_file in txt_files:
        print(f"Processing: {txt_file}")
        points = load_points_from_file(txt_file)
        if not points:
            print(f"  {txt_file} には点情報がありません。")
            continue

        # 座標を入れ替え＆固定キャリブレーション値に基づきスケーリング
        scaled_points = scale_points_swapped(points, width, draw_height)
        # フレーム番号ごとに点情報をグループ化
        frames_dict = {}
        for point in scaled_points:
            frame_num = point[0]
            frames_dict.setdefault(frame_num, []).append(point)
        frame_numbers = sorted(frames_dict.keys())
        
        base_filename = os.path.splitext(os.path.basename(txt_file))[0]
        
        # 元動画からFPSを取得
        original_video_path = f'../videos/Indoor/{base_filename}.mp4'
        cap = cv2.VideoCapture(original_video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        if fps <= 0:
            fps = 10  # 取得できなければデフォルト10
        print(f"Using FPS: {fps} for video {base_filename}")
        
        output_video_path = os.path.join(output_dir_GT, f'{base_filename}.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
        
        for frame_num in frame_numbers:
            frame_img = resized_image.copy()
            frame_points = frames_dict[frame_num]
            # 描画領域内に点と属性テキストを描画
            frame_img = draw_points(frame_img, frame_points)
            # フレーム番号も表示（フォントスケール1.0、太さ2）
            cv2.putText(frame_img, f"Frame: {frame_num}", (20, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
            video_writer.write(frame_img)
        
        video_writer.release()
        print(f"  動画が保存されました: {output_video_path}")

画像サイズ: width=1505, height=1050, 描画領域: 950px
Processing: ../ground_truth/Indoor/transformed_MOT_with_attributes/basket_S1T1_pre.txt
Using FPS: 19.98 for video basket_S1T1_pre
  動画が保存されました: ../videos/Indoor/minimap/ground_truth/basket_S1T1_pre.mp4
Processing: ../ground_truth/Indoor/transformed_MOT_with_attributes/basket_S1T2_pre.txt
Using FPS: 19.98 for video basket_S1T2_pre
  動画が保存されました: ../videos/Indoor/minimap/ground_truth/basket_S1T2_pre.mp4
Processing: ../ground_truth/Indoor/transformed_MOT_with_attributes/basket_S1T3_pre.txt
Using FPS: 19.98 for video basket_S1T3_pre
  動画が保存されました: ../videos/Indoor/minimap/ground_truth/basket_S1T3_pre.mp4
Processing: ../ground_truth/Indoor/transformed_MOT_with_attributes/basket_S1T4_pre.txt
Using FPS: 19.98 for video basket_S1T4_pre
  動画が保存されました: ../videos/Indoor/minimap/ground_truth/basket_S1T4_pre.mp4
Processing: ../ground_truth/Indoor/transformed_MOT_with_attributes/basket_S1T5_pre.txt
Using FPS: 19.98 for video basket_S1T5_pre
  動画が保存されました: ../v

In [ ]:
# 入力フォルダ設定（BoT-SORT側）
pred_dir = '../../BoT-SORT_outputs/Indoor/transformed/merged_with_attributes'
output_dir_pred = '../../videos/Indoor/minimap/prediction'
os.makedirs(output_dir_pred, exist_ok=True)

# ground_truth側のフォルダ（属性マッピング用）
gt_folder = '../../ground_truth/Indoor/transformed_MOT_with_attributes/'

# 背景画像の読み込みとリサイズ（強制的に縦1050×横1505）
base_image_path = '../../court_images/Indoor.jpg'
base_image = cv2.imread(base_image_path)
if base_image is None:
    print(f"背景画像の読み込みに失敗しました: {base_image_path}")
else:
    resized_image = cv2.resize(base_image, (1505, 1050))
    height, width = resized_image.shape[:2]  # height=1050, width=1505
    draw_height = height - 100  # 下部100px余白 → 描画領域は950px
    print(f"画像サイズ: width={width}, height={height}, 描画領域: {draw_height}px")
    
    txt_files_bot = glob.glob(os.path.join(pred_dir, '*.txt'))
    if not txt_files_bot:
        print("指定フォルダにtxtファイルが見つかりませんでした:", pred_dir)
    
    for txt_file in txt_files_bot:
        print(f"Processing: {txt_file}")
        points = load_points_from_file(txt_file)
        if not points:
            print(f"  {txt_file} には点情報がありません。")
            continue

        # 座標を入れ替え＆固定キャリブレーション値に基づきスケーリング
        scaled_points = scale_points_swapped(points, width, draw_height)
        # フレーム番号ごとに点情報をグループ化
        frames_dict = {}
        for point in scaled_points:
            frame_num = point[0]
            frames_dict.setdefault(frame_num, []).append(point)
        frame_numbers = sorted(frames_dict.keys())
        
        base_filename = os.path.splitext(os.path.basename(txt_file))[0]
        
        # 各BoT-SORTファイルに対応するground_truth側のファイルを探す
        gt_file = os.path.join(gt_folder, base_filename + '.txt')
        if os.path.exists(gt_file):
            attr_color_mapping = build_attribute_color_mapping_from_file(gt_file)
            print(f"Mapping for {base_filename}:", attr_color_mapping)
        else:
            print(f"ground_truth側に対応ファイルが見つかりません: {gt_file}")
            attr_color_mapping = {}
        
        # 元動画からFPSを取得（ファイル名は ./videos/Indoor/{base_filename}.mp4 と仮定）
        original_video_path = f'../../videos/Indoor/{base_filename}.mp4'
        cap = cv2.VideoCapture(original_video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        if fps <= 0:
            fps = 10  # 取得できなければデフォルト10
        print(f"Using FPS: {fps} for video {base_filename}")
        
        output_video_path = os.path.join(output_dir_pred, f'{base_filename}.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
        
        for frame_num in frame_numbers:
            frame_img = resized_image.copy()
            frame_points = frames_dict[frame_num]
            # 描画領域内に点と属性テキストを描画（属性マッピングを利用）
            frame_img = draw_points_with_mapping(frame_img, frame_points, attr_color_mapping)
            cv2.putText(frame_img, f"Frame: {frame_num}", (20, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
            video_writer.write(frame_img)
        
        video_writer.release()
        print(f"  動画が保存されました: {output_video_path}")


画像サイズ: width=1505, height=1050, 描画領域: 950px
Processing: ../BoT-SORT_outputs/Indoor/transformed/merged_with_attributes/basket_S6T3_post.txt
Mapping for basket_S6T3_post: {'O_top': (0, 0, 204), 'O_left': (204, 0, 0), 'O_right': (144, 84, 204), 'D_top': (0, 0, 0), 'D_left': (0, 120, 204), 'D_right': (108, 204, 176)}
Using FPS: 19.98 for video basket_S6T3_post
  動画が保存されました: ../videos/Indoor/minimap/prediction/basket_S6T3_post.mp4
Processing: ../BoT-SORT_outputs/Indoor/transformed/merged_with_attributes/basket_S6T4_post.txt
Mapping for basket_S6T4_post: {'O_top': (0, 0, 204), 'O_right': (204, 0, 0), 'O_left': (144, 84, 204), 'D_top': (0, 0, 0), 'D_right': (0, 120, 204), 'D_left': (108, 204, 176)}
Using FPS: 19.98 for video basket_S6T4_post
  動画が保存されました: ../videos/Indoor/minimap/prediction/basket_S6T4_post.mp4
Processing: ../BoT-SORT_outputs/Indoor/transformed/merged_with_attributes/basket_S6T5_post.txt
Mapping for basket_S6T5_post: {'O_top': (0, 0, 204), 'O_right': (204, 0, 0), 'O_left': (1